## Imports

In [45]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import requests
import time

## Data Read-in

In [2]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

print(len(df))

3461


In [3]:
gdf = gpd.read_file('countyboundary')

## Previous month's values

Get values from here:

https://docs.google.com/spreadsheets/d/1GWlqTIzCB29pDVDjd-iBVejVGGsLiAd0P6fq4LM83WI/edit?usp=sharing

In [4]:
print('Input County Name:')
county_name = input()

Input County Name:
Broward County


In [5]:
print('Input total condo units sold previous (not last) month:')
total_condos_sold_2nd_month = input()

Input total condo units sold previous (not last) month:
824


In [6]:
print('Input total sales volume ($$$ million) from previous (not last) month:')
sales_volume_2nd_month = input()

Input total sales volume ($$$ million) from previous (not last) month:
261000000


In [7]:
print('Input median sales price from previous (not last) month:')
median_sales_price_2nd_month = input()

Input median sales price from previous (not last) month:
235000


In [8]:
print('Input median psf from previous (not last) month:')
median_psf_2nd_month = input()

Input median psf from previous (not last) month:
232


## Data Clean

In [9]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [10]:
df = df.dropna(subset=['SOLD DATE'])

In [11]:
# Get the current date
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# Get the last day of the previous month by subtracting one day from the first day of the current month
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the month name of the previous month
previous_month_name = last_day_of_previous_month.strftime("%B")

# ---- Get the name of the month, as a string, from two months ago.
# ---- Meaning, if the current month is January, return November.

# Get the first day of the previous month by subtracting one day from the first day of the current month
first_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the first day of the month before the last by subtracting one day from the first day of the previous month
first_day_of_month_before_last = first_day_of_previous_month.replace(day=1) - timedelta(days=1)

# Get the month name of the month before the last
month_before_last_name = first_day_of_month_before_last.strftime("%B")

print(f'Report month: {previous_month_name} ---- Month before report month: {month_before_last_name}')

Report month: August ---- Month before report month: July


In [12]:
# Define list of desired months (excluding current month)
desired_months = [previous_month_name]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [13]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    965
Name: count, dtype: int64
-------
$/SQUARE FEET
False    961
True       4
Name: count, dtype: int64
-------
YEAR BUILT
False    965
Name: count, dtype: int64
-------


In [14]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/100-E-Las-Olas-Blvd-33301/unit-3701/home/147485697


In [15]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [16]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [17]:
df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)

In [18]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
29,PAST SALE,August-22-2024,Condo/Co-op,3690 Inverrary Dr Unit 1N,Lauderhill,FL,33319.0,122.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/3690-Inve...,Beaches MLS,F10446207,N,Y,26.171291,-80.230322
50,PAST SALE,August-15-2024,Condo/Co-op,4270 NW 89th Ave #101,Coral Springs,FL,33065.0,188.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Springs/4270-N...,MARMLS,A11576877,N,Y,26.282147,-80.245441
80,PAST SALE,August-26-2024,Condo/Co-op,4304 NW 9th Ave Unit 2-3B,Deerfield Beach,FL,33064.0,200.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/4304...,Beaches MLS,F10408643,N,Y,26.283701,-80.134083
22,PAST SALE,August-29-2024,Condo/Co-op,301 N Pine Island Rd #102,Plantation,FL,33324.0,260.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/301-N-Pin...,MARMLS,A11617580,N,Y,26.123358,-80.264762
28,PAST SALE,August-1-2024,Condo/Co-op,1470 N Dixie Hwy #33,Fort Lauderdale,FL,33304.0,21000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1470...,MARMLS,A11588584,N,Y,26.146156,-80.134920
4,PAST SALE,August-15-2024,Condo/Co-op,2705 NW 104 Ave #301,Sunrise,FL,33322.0,24300.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/2705-NW-104t...,MARMLS,A11576992,N,Y,26.159423,-80.288916
59,PAST SALE,August-23-2024,Condo/Co-op,1759 NW 81st Way #1759,Plantation,FL,33322.0,30000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/1759-NW-8...,MARMLS,A11549882,N,Y,26.145025,-80.260291
83,PAST SALE,August-29-2024,Condo/Co-op,9601 Sunrise Lakes Blvd #106,Sunrise,FL,33322.0,59000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/9601-Sunrise...,Beaches MLS,F10421054,N,Y,26.163650,-80.275854
77,PAST SALE,August-23-2024,Condo/Co-op,9480 Sunrise Lakes Blvd #306,Sunrise,FL,33322.0,60000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/9480-Sunrise...,Beaches MLS,F10427195,N,Y,26.159441,-80.278265
61,PAST SALE,August-30-2024,Condo/Co-op,8100 SW 24th St #310,North Lauderdale,FL,33068.0,60000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Lauderdale/810...,Beaches MLS,F10435611,N,Y,26.199049,-80.236998


In [28]:
print(df_filtered['URL'].iloc[61])

https://www.redfin.com/FL/North-Lauderdale/8100-SW-24th-St-33068/unit-310/home/41561916


In [29]:
# Correct the prices, if needed
df_filtered.at[29,'PRICE']=(122_000)
df_filtered.at[50,'PRICE']=(188_000)
df_filtered.at[80,'PRICE']=(200_000)
df_filtered.at[22,'PRICE']=(260_000)
df_filtered.at[28,'PRICE']=(210_000)
df_filtered.at[4,'PRICE']=(243_000)
df_filtered.at[59,'PRICE']=(300_000)

# Correct the psf, if needed
df_filtered.at[29,'$/SQUARE FEET']=(122_000/800)
df_filtered.at[50,'$/SQUARE FEET']=(188_000/1070)
df_filtered.at[80,'$/SQUARE FEET']=(200_000/1290)
df_filtered.at[22,'$/SQUARE FEET']=(260_000/1275)
df_filtered.at[28,'$/SQUARE FEET']=(210_000/733)
df_filtered.at[4,'$/SQUARE FEET']=(243_000/1370)
df_filtered.at[59,'$/SQUARE FEET']=(300_000/1034)

In [61]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [24]:
# print(df_filtered['URL'].iloc[528])

In [30]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
51,72500.0,8100 SW 24th St #112,North Lauderdale,56.0
61,60000.0,8100 SW 24th St #310,North Lauderdale,71.0
3,65000.0,5750 NW 64th Ave #304,Tamarac,74.0
48,75000.0,2061 NW 47th Ter #214,Lauderhill,81.0
30,90000.0,2850 Somerset Dr Unit 309L,Lauderdale Lakes,89.0
38,95000.0,3571 Inverrary Dr #103,Lauderhill,89.0
292,130000.0,6200 S Falls Circle Dr #108,Lauderhill,90.0
83,59000.0,9601 Sunrise Lakes Blvd #106,Sunrise,92.0
49,70000.0,9351 Lime Bay Blvd #209,Tamarac,93.0
52,80000.0,10303 Sunrise Lakes Blvd #206,Sunrise,93.0


In [63]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [31]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [32]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [33]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [34]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [35]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [36]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{previous_month_name} 2024 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(gdf,tooltip=f'{county_name}',name=f'{county_name}').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [37]:
m.save('index.html')

## Summary Info

In [38]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [39]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [40]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [41]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/1060-SE-14th-Pl-33316/unit-16D/home/41806742


In [42]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Four Seasons, 525 N Ft Lauderdale Beach Blvd #801 Fort Lauderdale | Price $4,750,000 | $2,339 psf | Year built: 2022
Least Expensive
Sunrise Lakes Phase 3, 9601 Sunrise Lakes Blvd #106 Sunrise | Price $59,000 | $92 psf | Year built: 1980
Highest Price Per Square Foot
Four Seasons, 525 N Ft Lauderdale Beach Blvd #801 Fort Lauderdale | Price $4,750,000 | $2,339 psf | Year built: 2022
Lowest Price Per Square Foot
Oakbrook, 8100 SW 24th St #112 North Lauderdale | Price $72,500 | $56 psf | Year built: 1973
Newest
Four Seasons, 525 N Ft Lauderdale Beach Blvd #801 Fort Lauderdale | Price $4,750,000 | $2,339 psf | Year built: 2022
Oldest
Cliff Lake Villas, 1060 SE 14th Pl Unit 16D Fort Lauderdale | Price $177,000 | $221 psf | Year built: 1958


## Map URL Snagger

In [43]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/BC_condo_sales_month_ending_august_2024


## Get Summary Data

In [44]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 824
--------
Total sale price: $252,801,246
--------
Median sale price: $226,250
--------
Max sale price: $4,750,000
--------
Min sale price: $59,000
------------------------------------------------
PSF INFO
Max price per square foot: $2,339
--------
Min price per square foot: $56
--------
Median price per square foot: $224
------------------------------------------------
CONDO AGES
Newest building: 2022.0
----------
Oldest building: 1958.0
----------
Average building age: 1980.2014563106795


## Collect Agent Data

In [46]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [47]:
df_filtered.reset_index(inplace=True,drop=True)

In [48]:
max_price_index = df_filtered['PRICE'].idxmax()
min_price_index = df_filtered['PRICE'].idxmin()

max_price_psf_index = df_filtered['$/SQUARE FEET'].idxmax()
min_price_psf_index = df_filtered['$/SQUARE FEET'].idxmin()

max_year_built_index = df_filtered['YEAR BUILT'].idxmax()
min_year_built_index = df_filtered['YEAR BUILT'].idxmin()

In [49]:
index_list = [max_price_index,min_price_index,
             max_price_psf_index,min_price_psf_index,
             max_year_built_index,min_year_built_index]

In [50]:
response_list = []

for index, row in df_filtered.iterrows():
    if index in index_list:
        response_list.append(process_response(row['URL']))
        
response_df = pd.DataFrame(response_list)

merged_df = pd.merge(left=df_filtered,left_on='URL',right=response_df, right_on='url',how='outer')

In [51]:
### Test to see if data was collected
merged_df['list_agent_name_1'].unique()

array([nan, "Shannon O'Brien", 'Nicholas Foley', 'David Mohabir',
       'Mervin Moss'], dtype=object)

In [52]:
df_filtered = merged_df

In [53]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")

Most Expensive
Four Seasons, 525 N Ft Lauderdale Beach Blvd #801 Fort Lauderdale | Price $4,750,000 | $2,339 psf | Year built: 2022 | Listing agents: Nicholas Foley with Compass Florida, LLC. and None with None | Buyer's agents: Robin Gervais with Best of Luxury Realty Corp. and Tracey Schmitt with Best of Luxury Realty Corp.
Least Expensive
Sunrise Lakes Phase 3, 9601 Sunrise Lakes Blvd #106 Sunrise | Price $59,000 | $92 psf | Year built: 1980 | Listing agents: Mervin Moss with RE/MAX Direct and None with None | Buyer's agents: Veronica Paladines with Dezer Platinum Realty LLC and None with None
Highest Price Per Square Foot
Four Seasons, 525 N Ft Lauderdale Beach Blvd #801 Fort Lauderdale | Price $4,750,000 | $2,339 psf | Year built: 2022 | Listing agents: Nicholas Foley with Compass Florida, LLC. and None with None | Buyer's agents: Robin Gervais with Best of Luxury Realty Corp. and Tracey Schmitt with Best of Luxury Realty Corp.
Lowest Price Per Square Foot
Oakbrook, 8100 SW 24th S

## Get condo names and locations for most and least expensive

In [54]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

In [55]:
## Calculate the increase/descrease between both months
subject_month_sales_volume = df_filtered["PRICE"].sum()
prior_month_sales_volume = int(sales_volume_2nd_month)  # Example value, replace with actual value

subject_month_closings = len(df_filtered)
prior_month_closings = int(total_condos_sold_2nd_month)

# Subtract the smaller value from the larger one
if subject_month_sales_volume > prior_month_sales_volume:
    hed_rf = 'jumps'
    nut_graf = 'rising'
    second_nut_graf_ref = 'up'
    social = 'higher'
    seo = 'increased'
elif prior_month_sales_volume > subject_month_sales_volume:
    hed_rf = 'drops'
    nut_graf = 'falling'
    second_nut_graf_ref = 'down'
    social = 'lower'
    seo = 'dropped'
else:
    hed_rf = 'equals'
    nut_graf = 'equaling'
    second_nut_graf_ref = 'on par with'
    social = 'equal'
    seo = "did't change"
    
    
if subject_month_closings > prior_month_closings:
    dek_rf = 'rose'
elif prior_month_closings > subject_month_closings:
    dek_rf ='fell'
else:
    dek_rf = 'equaled'

In [56]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS', 'url',
       'list_agent_name_1', 'list_broker_name_1', 'list_agent_name_2',
       'list_broker_name_2', 'buy_agent_name_1', 'buy_broker_name_1',
       'buy_agent_name_2', 'buy_broker_name_2'],
      dtype='object')

In [57]:
# Get values for the top sale (max price)
top_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_1']
top_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_1']
top_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_2']
top_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_2']
top_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_1']
top_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_1']
top_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_2']
top_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_2']

# Get values for the bottom sale (min price)
bottom_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_1']
bottom_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_1']
bottom_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_2']
bottom_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_2']
bottom_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_1']
bottom_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_1']
bottom_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_2']
bottom_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_2']

# Now create the formatted string
test_string = f'''


TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

'''

print(test_string)





TOP SALE LISTING AGENTS:

Nicholas Foley with Compass Florida, LLC. and None with None

TOP SALE BUYER AGENTS:

Robin Gervais with Best of Luxury Realty Corp. and Tracey Schmitt with Best of Luxury Realty Corp.

BOTTOM SALE LISTING AGENTS:

Mervin Moss with RE/MAX Direct and None with None

BOTTOM SALE BUYER AGENTS:

Veronica Paladines with Dezer Platinum Realty LLC and None with None




In [58]:
story_string = f'''
\033[1mHED:\033[0m {county_name} condo sales dollar volume {hed_rf} in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings {dek_rf} to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county_name} {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m {county_name}’s condo sales volume {seo} to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m  
\033[1mSocial:\033[0m {county_name} had ${round(df_filtered["PRICE"].sum()/1_000_000):,.0f} million in condo sales in {previous_month_name}, {social} than ${round(int(sales_volume_2nd_month)/1_000_000)} million from {month_before_last_name}, @afarence reports 
\033[1mART:\033[0m 

CHANGE ME

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county_name}, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m

{county_name}’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${round(df_filtered["PRICE"].sum()/1_000_000)} million, {nut_graf} from ${int(sales_volume_2nd_month)/1_000_000:.0f} million in {month_before_last_name}. Brokers closed {len(df_filtered)} sales last month, {second_nut_graf_ref} from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, Multiple Listing Service data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.0f} to ${df_filtered["PRICE"].max():,.0f} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged from ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${int(median_sales_price_2nd_month):,.0f}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max():,.0f} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} sold for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. Unit XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min():,.0f} — or ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [59]:
print(story_string)

print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")


HED: Broward County condo sales dollar volume drops in August to $253M 
DEK: Number of closings equaled to 824 from 824 in July
FEATURED HED:
SEO HED: Broward County August Condo Sales Report 
SEO DESCRIPTION: Broward County’s condo sales volume dropped to $253 million in August.
AUTHOR: Adam Farence
RESEARCH:  
Social: Broward County had $253 million in condo sales in August, lower than $261 million from July, @afarence reports 
ART: 

CHANGE ME

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, monthly condo sales, condos, condo market, Four Seasons, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:

Broward County’s NEWS PEG HERE.

August condo sales totaled $253 million, falling from $261 million in July. Brokers closed 824 sales last month, down from 824 sales in July and XXX in XXX, Multiple Listing Service data from Redfin show

# Print links for notable sales

## Top sale

In [60]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/525-N-Fort-Lauderdale-Beach-Blvd-33304/unit-801/home/116680919


## Cheapest sale

In [66]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Sunrise/9601-Sunrise-Lakes-Blvd-33322/unit-106/home/41606477


## Highest PSF

In [67]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/525-N-Fort-Lauderdale-Beach-Blvd-33304/unit-801/home/116680919


## Lowest PSF

In [68]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/North-Lauderdale/8100-SW-24th-St-33068/unit-112/home/41562495


## Newest

In [69]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/525-N-Fort-Lauderdale-Beach-Blvd-33304/unit-801/home/116680919


## Oldest

In [70]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/1060-SE-14th-Pl-33316/unit-16D/home/41806742


## Time on Market Calculator

In [72]:
from datetime import datetime, timedelta

################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 2, 2) ## List (Earlier) date
date2 = datetime(2024, 8, 29) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

209


## Access second highest (or whatever is needed)

In [58]:
# ### Second highest ###
# # Sort the DataFrame by '$/SQUARE FEET' in descending order
# second_highest = df_filtered.sort_values(by='$/SQUARE FEET', ascending=False)

# # Get the URL of the entry with the second highest '$/SQUARE FEET' value
# second_highest_url = second_highest.iloc[1]['URL']  # `.iloc[1]` accesses the second row

# print(second_highest_url)